# Поиск по заглавиям романов

## инсталы

In [1]:
!pip install datasets transformers scikit-learn nltk pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.2 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=930326e8a26f1b23139db6eee98ee092bfe0339317ba1a0356130acd9e1efe38
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
  Attempting uninstall: pyarrow
    Fo

In [4]:
import numpy as np
import pandas as pd
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch
import nltk
import pymorphy2
from tqdm import tqdm
import time
import torch
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
from tqdm import tqdm

In [95]:
url = 'https://raw.githubusercontent.com/dachelnokova/rus_novel_title/main/rus_novel_titles.csv'
data = pd.read_csv(url)

# предобработка текста
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
morph = pymorphy2.MorphAnalyzer()
russian_stopwords = set(stopwords.words('russian'))

def preprocess_text(text):
    words = nltk.word_tokenize(text.lower())
    words = [morph.parse(word)[0].normal_form for word in words if word.isalnum()]
    words = [word for word in words if word not in russian_stopwords]
    return ' '.join(words)

# подготовка данных
data['processed_title'] = data['title'].apply(preprocess_text)
data[['title', 'processed_title']].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,title,processed_title
0,"Награждённая постоянность, или Приключения Лиз...",наградить постоянность приключение лизарка сар...
1,"Непостоянная фортуна, или Похождения Мирамонда",непостоянный фортуна похождение мирамонд
2,"Приключения Фемистокла и разные политические, ...",приключение фемистокнуть разный политический г...
3,"Пересмешник, или Славенские сказки",пересмешник славенский сказка
4,Письма Эрнеста и Доравры,письмо эрнест доравр


## 1. Полнотекстовый поиск (Tfidf)

In [97]:
# 1. полнотекстовый поиск
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data['processed_title'])

def full_text_search(query, top_k=5):
    processed_query = preprocess_text(query)
    query_vec = vectorizer.transform([processed_query])
    similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_indices = similarities.argsort()[-top_k:][::-1]
    return [(data.iloc[i]['title'], similarities[i]) for i in top_indices]

In [ ]:
# import joblib
# joblib.dump(tfidf_matrix, 'tfidf_matrix.pkl')
# joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

## 2. Векторный поиск (sberbank-ai/sbert_large_nlu_ru)

In [98]:
# 2. векторный поиск
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("sberbank-ai/sbert_large_nlu_ru")

In [99]:
import torch

In [100]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(120138, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1,

In [101]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

In [102]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np

In [104]:
from tqdm import tqdm

In [103]:
def compute_embedding(text):
    return get_embedding(text)

In [65]:
# создание эмбеддингов для всех документов
from multiprocessing import Pool, cpu_count

def create_document_embeddings(texts, processes=None):
    with Pool(processes=processes or cpu_count()) as pool:
        document_embeddings = list(tqdm(pool.imap(compute_embedding, texts), total=len(texts), desc="Processing embeddings"))
    return np.array(document_embeddings)

# gpu
# def create_document_embeddings(texts, max_workers=4):
#     document_embeddings = []

#     with ProcessPoolExecutor(max_workers=max_workers) as executor:
#         futures = [executor.submit(compute_embedding, text) for text in texts]
#         for future in tqdm(as_completed(futures), total=len(futures), desc="Processing embeddings"):
#             document_embeddings.append(future.result())

#     return np.array(document_embeddings)


# вызов функции с данными
document_embeddings = create_document_embeddings(data['title'])

Processing embeddings: 100%|██████████| 2036/2036 [10:39<00:00,  3.19it/s]


In [66]:
document_embeddings

array([[ 1.001265  , -0.46314391, -0.23325287, ...,  0.02570524,
         0.6384671 , -0.04932671],
       [ 0.7336704 , -0.4700265 , -0.26066113, ...,  0.22309197,
         0.34124857, -0.13451265],
       [ 0.7241792 , -0.40413648, -0.8195384 , ..., -0.10955931,
         0.9635812 , -0.1345805 ],
       ...,
       [ 0.470502  , -0.1615254 , -0.46566772, ..., -0.28230423,
        -0.64112973,  0.5520886 ],
       [ 0.6015674 , -0.38889933, -0.8934784 , ..., -0.2547407 ,
        -0.00893512, -0.03332439],
       [ 0.08298261, -0.2955161 , -0.81802297, ..., -0.18663372,
        -0.4014135 ,  0.19867627]], dtype=float32)

In [68]:
# нормализация эмбеддингов
from sklearn.preprocessing import normalize

normalized_embeddings = normalize(document_embeddings)

In [69]:
normalized_embeddings

array([[ 0.06239568, -0.02886167, -0.01453558, ...,  0.00160187,
         0.03978725, -0.00307388],
       [ 0.04491978, -0.02877789, -0.01595926, ...,  0.01365905,
         0.02089332, -0.00823569],
       [ 0.04140471, -0.02310637, -0.04685684, ..., -0.00626402,
         0.05509244, -0.0076946 ],
       ...,
       [ 0.027622  , -0.00948276, -0.0273382 , ..., -0.01657338,
        -0.03763913,  0.03241175],
       [ 0.03837259, -0.02480699, -0.05699291, ..., -0.01624932,
        -0.00056995, -0.00212569],
       [ 0.00535637, -0.01907502, -0.05280187, ..., -0.01204686,
        -0.0259105 ,  0.01282419]], dtype=float32)

In [77]:
np.save('sbert_large_nlu_ru_embeddings.npy', normalized_embeddings)

In [105]:
sbert_embeddings = np.load('sbert_large_nlu_ru_embeddings.npy')

In [ ]:
# !ls /content

In [106]:
def vector_search(query, top_k=5):
      # используем оригинальный запрос без предобработки
    query_vec = get_embedding(query)
    similarities = cosine_similarity([query_vec], sbert_embeddings).flatten()
    top_indices = similarities.argsort()[-top_k:][::-1]
    return [(data.iloc[i]['title'], similarities[i]) for i in top_indices]


## 3. Гибридный поиск

In [107]:
# 3. гибридный поиск
def hybrid_search(query, top_k=5, weight_full_text=0.5):
    full_text_results = full_text_search(query, top_k)
    vector_results = vector_search(query, top_k)

    combined_results = {}
    for doc, score in full_text_results:
        combined_results[doc] = weight_full_text * score

    for doc, score in vector_results:
        if doc in combined_results:
            combined_results[doc] += (1 - weight_full_text) * score
        else:
            combined_results[doc] = (1 - weight_full_text) * score

    sorted_results = sorted(combined_results.items(), key=lambda x: x[1], reverse=True)
    return sorted_results[:top_k]

## Проверка как работает

In [108]:
def perform_search(query, top_k, method):
    start_time = time.time()

    if method == 'full_text':
        results = full_text_search(query, top_k)
    elif method == 'vector':
        results = vector_search(query, top_k)
    elif method == 'hybrid':
        results = hybrid_search(query, top_k)
    else:
        raise ValueError("Invalid search method. Choose from 'full_text', 'vector', 'hybrid'.")

    end_time = time.time()
    elapsed_time = end_time - start_time

    # нумерация результатов
    numbered_results = [(i + 1, doc, score) for i, (doc, score) in enumerate(results)]

    print(f"\nВаш поисковой запрос: {query}")
    print(f"Выбранный метод поиска: {method}")
    print(f"Количество результатов для выдачи: {top_k}")
    print(f"Время исполнения: {elapsed_time:.4f} секунд")
    print("-" * 34)

    for i, doc, score in numbered_results:
        print(f"{i}. Скор:  {score:.4f}")
        print(f"Заглавие: {doc}\n")


### Отцы и дети

In [22]:
perform_search("Отцы и дети", 5, 'full_text')


Ваш поисковой запрос: Отцы и дети
Выбранный метод поиска: full_text
Количество результатов для выдачи: 5
Время исполнения: 0.0214 секунд
----------------------------------
1. Скор:  1.0000
Заглавие: Отцы и дети

2. Скор:  0.7284
Заглавие: Дети

3. Скор:  0.5384
Заглавие: Дети земли

4. Скор:  0.4754
Заглавие: Нежеланные дети

5. Скор:  0.4694
Заглавие: За грехи отца



In [23]:
perform_search("Отцы и дети", 5, 'vector')


Ваш поисковой запрос: Отцы и дети
Выбранный метод поиска: vector
Количество результатов для выдачи: 5
Время исполнения: 3.0316 секунд
----------------------------------
1. Скор:  1.0000
Заглавие: Отцы и дети

2. Скор:  0.8550
Заглавие: Мать и дочь

3. Скор:  0.8415
Заглавие: Мужья и жёны

4. Скор:  0.8145
Заглавие: Дед и внук

5. Скор:  0.8087
Заглавие: Дети



In [24]:
perform_search("Отцы и дети", 5, 'hybrid')


Ваш поисковой запрос: Отцы и дети
Выбранный метод поиска: hybrid
Количество результатов для выдачи: 5
Время исполнения: 0.3202 секунд
----------------------------------
1. Скор:  1.0000
Заглавие: Отцы и дети

2. Скор:  0.7685
Заглавие: Дети

3. Скор:  0.4275
Заглавие: Мать и дочь

4. Скор:  0.4208
Заглавие: Мужья и жёны

5. Скор:  0.4073
Заглавие: Дед и внук



### мяу

In [30]:
perform_search("мяу", 5, 'full_text')


Ваш поисковой запрос: мяу
Выбранный метод поиска: full_text
Количество результатов для выдачи: 5
Время исполнения: 0.0058 секунд
----------------------------------
1. Скор:  0.0000
Заглавие: Моя весна

2. Скор:  0.0000
Заглавие: Идеалисты и реалисты

3. Скор:  0.0000
Заглавие: Царский суд

4. Скор:  0.0000
Заглавие: Мещане

5. Скор:  0.0000
Заглавие: Убийца



In [31]:
perform_search("мяу", 5, 'vector')


Ваш поисковой запрос: мяу
Выбранный метод поиска: vector
Количество результатов для выдачи: 5
Время исполнения: 0.2660 секунд
----------------------------------
1. Скор:  0.6423
Заглавие: Муся

2. Скор:  0.6285
Заглавие: Мура

3. Скор:  0.6238
Заглавие: Мимочка

4. Скор:  0.5934
Заглавие: Котик Летаев

5. Скор:  0.5784
Заглавие: Мыканье



In [33]:
perform_search("мяу", 5, 'hybrid')


Ваш поисковой запрос: мяу
Выбранный метод поиска: hybrid
Количество результатов для выдачи: 5
Время исполнения: 0.3224 секунд
----------------------------------
1. Скор:  0.3211
Заглавие: Муся

2. Скор:  0.3142
Заглавие: Мура

3. Скор:  0.3119
Заглавие: Мимочка

4. Скор:  0.2967
Заглавие: Котик Летаев

5. Скор:  0.2892
Заглавие: Мыканье



### уют

In [73]:
perform_search("уют", 5, 'full_text')

Выполняется полнотекстовый поиск...
Предобработка текста: уют
Полнотекстовый поиск завершен.

Ваш поисковой запрос: уют
Выбранный метод поиска: full_text
Количество результатов для выдачи: 5
Время исполнения: 0.0193 секунд
----------------------------------
1. Скор:  0.0000
Заглавие: Моя весна

2. Скор:  0.0000
Заглавие: Идеалисты и реалисты

3. Скор:  0.0000
Заглавие: Царский суд

4. Скор:  0.0000
Заглавие: Мещане

5. Скор:  0.0000
Заглавие: Убийца



In [72]:
perform_search("уют", 5, 'vector')


Ваш поисковой запрос: уют
Выбранный метод поиска: vector
Количество результатов для выдачи: 5
Время исполнения: 0.3845 секунд
----------------------------------
1. Скор:  0.7620
Заглавие: Современная идиллия

2. Скор:  0.7534
Заглавие: Природа

3. Скор:  0.7462
Заглавие: Кадм и Гармония

4. Скор:  0.7375
Заглавие: Счастье

5. Скор:  0.7367
Заглавие: Красота



In [75]:
perform_search("уют", 5, 'hybrid')

Выполняется гибридный поиск...
Выполняется полнотекстовый поиск...
Предобработка текста: уют
Полнотекстовый поиск завершен.
Гибридный поиск завершен.

Ваш поисковой запрос: уют
Выбранный метод поиска: hybrid
Количество результатов для выдачи: 5
Время исполнения: 0.3413 секунд
----------------------------------
1. Скор:  0.3810
Заглавие: Современная идиллия

2. Скор:  0.3767
Заглавие: Природа

3. Скор:  0.3731
Заглавие: Кадм и Гармония

4. Скор:  0.3687
Заглавие: Счастье

5. Скор:  0.3684
Заглавие: Красота



# Полный код запуска

In [81]:
import argparse
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch
import nltk
import pymorphy2
from tqdm import tqdm
import time
import joblib

# загрузка необходимых ресурсов для обработки текста
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

# инициализация морфологического анализатора и списка стоп-слов для русского
morph = pymorphy2.MorphAnalyzer()
russian_stopwords = set(stopwords.words('russian'))

def preprocess_text(text):
    """
    Функция для предобработки текста:
    1. Приведение к нижнему регистру.
    2. Токенизация.
    3. Нормализация слов с использованием морфологического анализатора.
    4. Удаление стоп-слов.

    :param text: Строка текста для обработки.
    :return: Предобработанный текст.
    """
    print(f"Предобработка текста: {text}")
    words = nltk.word_tokenize(text.lower())
    words = [morph.parse(word)[0].normal_form for word in words if word.isalnum()]
    words = [word for word in words if word not in russian_stopwords]
    return ' '.join(words)

def full_text_search(query, top_k=5):
    """
    Полнотекстовый поиск с использованием TF-IDF.

    :param query: Поисковый запрос.
    :param top_k: Количество возвращаемых результатов.
    :return: Список кортежей (заголовок, оценка сходства).
    """
    print("Выполняется полнотекстовый поиск...")
    processed_query = preprocess_text(query)
    query_vec = vectorizer.transform([processed_query])
    similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_indices = similarities.argsort()[-top_k:][::-1]
    print("Полнотекстовый поиск завершен.")
    return [(data.iloc[i]['title'], similarities[i]) for i in top_indices]

def get_embedding(text):
    """
    Получение эмбеддинга текста с использованием модели sberbank-ai/sbert_large_nlu_ru.

    :param text: Текст для преобразования в эмбеддинг.
    :return: Эмбеддинг текста в виде numpy массива.
    """
    print(f"Получение эмбеддинга для: {text}")
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        return_tensors="pt",
        truncation=True,
        padding='max_length',
        max_length=512
    )
    inputs = inputs.to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    print("Эмбеддинг получен.")
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

def vector_search(query, top_k=5):
    """
    Векторный поиск с использованием эмбеддингов.

    :param query: Поисковый запрос.
    :param top_k: Количество возвращаемых результатов.
    :return: Список кортежей (заголовок, оценка сходства).
    """
    print("Выполняется векторный поиск...")
    query_vec = get_embedding(query)
    similarities = cosine_similarity([query_vec], sbert_embeddings).flatten()
    top_indices = similarities.argsort()[-top_k:][::-1]
    print("Векторный поиск завершен.")
    return [(data.iloc[i]['title'], similarities[i]) for i in top_indices]

def hybrid_search(query, top_k=5, weight_full_text=0.5):
    """
    Гибридный поиск, комбинирующий полнотекстовый и векторный подходы.

    :param query: Поисковый запрос.
    :param top_k: Количество возвращаемых результатов.
    :param weight_full_text: Вес для полнотекстового поиска в гибридной модели.
    :return: Список кортежей (заголовок, комбинированная оценка сходства).
    """
    print("Выполняется гибридный поиск...")
    full_text_results = full_text_search(query, top_k)
    vector_results = vector_search(query, top_k)

    combined_results = {}
    for doc, score in full_text_results:
        combined_results[doc] = weight_full_text * score

    for doc, score in vector_results:
        if doc in combined_results:
            combined_results[doc] += (1 - weight_full_text) * score
        else:
            combined_results[doc] = (1 - weight_full_text) * score

    sorted_results = sorted(combined_results.items(), key=lambda x: x[1], reverse=True)
    print("Гибридный поиск завершен.")
    return sorted_results[:top_k]

def perform_search(query, top_k, method):
    """
    Основная функция для выполнения поиска по выбранному методу.

    :param query: Поисковый запрос.
    :param top_k: Количество возвращаемых результатов.
    :param method: Метод поиска ('full_text', 'vector', 'hybrid').
    """
    start_time = time.time()

    if method == 'full_text':
        results = full_text_search(query, top_k)
    elif method == 'vector':
        results = vector_search(query, top_k)
    elif method == 'hybrid':
        results = hybrid_search(query, top_k)
    else:
        raise ValueError("Неверный метод поиска. Выберите из 'full_text', 'vector', 'hybrid'.")

    end_time = time.time()
    elapsed_time = end_time - start_time

    # нумерация результатов и вывод
    numbered_results = [(i + 1, doc, score) for i, (doc, score) in enumerate(results)]

    print(f"\nВаш поисковой запрос: {query}")
    print(f"Выбранный метод поиска: {method}")
    print(f"Количество результатов для выдачи: {top_k}")
    print(f"Время исполнения: {elapsed_time:.4f} секунд")
    print("-" * 34)

    for i, doc, score in numbered_results:
        print(f"{i}. Скор:  {score:.4f}")
        print(f"Заглавие: {doc}\n")

print("Загрузка данных и моделей...")
data = pd.read_csv('rus_novel_titles.csv')

tfidf_matrix = joblib.load('tfidf_matrix.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')
sbert_embeddings = np.load('sbert_large_nlu_ru_embeddings.npy')

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("sberbank-ai/sbert_large_nlu_ru")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Данные и модели загружены.")

# if __name__ == "__main__":
#     parser = argparse.ArgumentParser(description='Поиск')
#     parser.add_argument('--query', type=str, required=True, help='Поисковый запрос')
#     parser.add_argument('--count', type=int, default=5, help='Количество возвращаемых результатов')
#     parser.add_argument('--method', type=str, default='hybrid', choices=['full_text', 'vector', 'hybrid'], help='Метод поиска')
#     args = parser.parse_args()

#     perform_search(args.query, args.count, args.method)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Загрузка данных и моделей...
Данные и модели загружены.


In [ ]:
perform_search("самолет", 5, 'full_text')


Ваш поисковой запрос: самолет
Выбранный метод поиска: full_text
Количество результатов для выдачи: 5
Время исполнения: 0.0306 секунд
----------------------------------
1. Скор:  0.0000
Заглавие: Моя весна

2. Скор:  0.0000
Заглавие: Идеалисты и реалисты

3. Скор:  0.0000
Заглавие: Царский суд

4. Скор:  0.0000
Заглавие: Мещане

5. Скор:  0.0000
Заглавие: Убийца



In [ ]:
perform_search("самолет", 5, 'vector')


Ваш поисковой запрос: самолет
Выбранный метод поиска: vector
Количество результатов для выдачи: 5
Время исполнения: 0.3394 секунд
----------------------------------
1. Скор:  0.6831
Заглавие: Ветер

2. Скор:  0.6816
Заглавие: Метеор

3. Скор:  0.6807
Заглавие: Крылья

4. Скор:  0.6702
Заглавие: Автомат

5. Скор:  0.6557
Заглавие: Капитон Перелетов



In [ ]:
perform_search("самолет", 5, 'hybrid')

Выполняется гибридный поиск...
Выполняется полнотекстовый поиск...
Предобработка текста: самолет
Полнотекстовый поиск завершен.
Выполняется векторный поиск...
Получение эмбеддинга для: самолет
Эмбеддинг получен.
Векторный поиск завершен.
Гибридный поиск завершен.

Ваш поисковой запрос: самолет
Выбранный метод поиска: hybrid
Количество результатов для выдачи: 5
Время исполнения: 10.5299 секунд
----------------------------------
1. Скор:  0.3023
Заглавие: Капитон Перелетов

2. Скор:  0.2759
Заглавие: Новая метла

3. Скор:  0.2717
Заглавие: Метеор

4. Скор:  0.2685
Заглавие: Крылья

5. Скор:  0.2657
Заглавие: Корнет Отлетаев

